In [ ]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers

# ========================
# 1. Hyperparameters and Configuration
# ========================

# Set random seed for reproducibility
manualSeed = 999
tf.random.set_seed(manualSeed)
np.random.seed(manualSeed)

# Hyperparameters
dataroot = 'data'
batch_size = 128
image_size = 64  # Resize CIFAR-10 images to 64x64
nc = 3          # Number of channels in the training images (CIFAR-10 is RGB)
nz = 100        # Size of z latent vector (i.e., size of generator input)
ngf = 64        # Size of feature maps in generator
ndf = 64        # Size of feature maps in discriminator
num_epochs = 50
lr = 0.0001
beta1 = 0.5     # Beta1 hyperparam for Adam optimizers

# Create output directory
os.makedirs('dcgan_results', exist_ok=True)

# ========================
# 2. Data Loading and Preprocessing
# ========================

# Load the dataset
(train_images, _), (_, _) = tf.keras.datasets.cifar10.load_data()

# Preprocess the data
def preprocess(image):
    image = tf.cast(image, tf.float32)  # Cast to float32
    # Resize to 64x64
    image = tf.image.resize(image, [image_size, image_size])
    # Normalize to [-1, 1]
    image = (image - 127.5) / 127.5
    return image

# Create a tf.data.Dataset and apply preprocessing
train_dataset = tf.data.Dataset.from_tensor_slices(train_images)
train_dataset = train_dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.shuffle(50000).batch(batch_size)
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

# ========================
# 3. Define the Generator Network
# ========================

def make_generator_model():
    model = tf.keras.Sequential()
    # Input is Z, going into a dense layer
    model.add(layers.Dense(4*4*ngf*8, use_bias=False, input_shape=(nz,)))
    model.add(layers.Reshape((4, 4, ngf*8)))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    # (4x4xngf*8) -> (8x8xngf*4)
    model.add(layers.Conv2DTranspose(ngf*4, kernel_size=4, strides=2, padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    # (8x8xngf*4) -> (16x16xngf*2)
    model.add(layers.Conv2DTranspose(ngf*2, kernel_size=4, strides=2, padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    # (16x16xngf*2) -> (32x32xngf)
    model.add(layers.Conv2DTranspose(ngf, kernel_size=4, strides=2, padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    # (32x32xngf) -> (64x64xnc)
    model.add(layers.Conv2DTranspose(nc, kernel_size=4, strides=2, padding='same', use_bias=False, activation='tanh'))

    return model

# Instantiate the generator
generator = make_generator_model()

# ========================
# 4. Define the Discriminator Network
# ========================

def make_discriminator_model():
    model = tf.keras.Sequential()
    # Input is (64x64xnc)
    model.add(layers.Conv2D(ndf, kernel_size=4, strides=2, padding='same', input_shape=[image_size, image_size, nc]))
    model.add(layers.LeakyReLU(alpha=0.2))

    # (32x32xndf) -> (16x16xndf*2)
    model.add(layers.Conv2D(ndf*2, kernel_size=4, strides=2, padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))

    # (16x16xndf*2) -> (8x8xndf*4)
    model.add(layers.Conv2D(ndf*4, kernel_size=4, strides=2, padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))

    # (8x8xndf*4) -> (4x4xndf*8)
    model.add(layers.Conv2D(ndf*8, kernel_size=4, strides=2, padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))

    # Output layer
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))

    return model

# Instantiate the discriminator
discriminator = make_discriminator_model()

# ========================
# 5. Loss Function and Optimizers
# ========================

# Loss function
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=False)

# Optimizers
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=lr, beta_1=beta1)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=lr, beta_1=beta1)

# Fixed noise vector for generating images
fixed_noise = tf.random.normal([64, nz])

# ========================
# 6. Training Loop
# ========================

# Lists to keep track of progress
G_losses = []
D_losses = []
img_list = []
iters = 0

print("Starting Training Loop...")

# Training step function
@tf.function
def train_step(images):
    batch_size = tf.shape(images)[0]
    noise = tf.random.normal([batch_size, nz])

    # Label smoothing
    real_labels = tf.ones((batch_size, 1)) * 0.9  # Real label smoothing
    fake_labels = tf.zeros((batch_size, 1))

    # ---------------------
    # Train Discriminator
    # ---------------------
    with tf.GradientTape() as disc_tape:
        # Generate fake images
        generated_images = generator(noise, training=True)

        # Get discriminator outputs
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        # Calculate losses
        d_loss_real = cross_entropy(real_labels, real_output)
        d_loss_fake = cross_entropy(fake_labels, fake_output)
        d_loss = d_loss_real + d_loss_fake

    # Compute gradients and update discriminator
    gradients_of_discriminator = disc_tape.gradient(d_loss, discriminator.trainable_variables)
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    # -----------------
    # Train Generator
    # -----------------
    with tf.GradientTape() as gen_tape:
        generated_images = generator(noise, training=True)
        fake_output = discriminator(generated_images, training=True)

        # Generator wants discriminator to think images are real
        g_loss = cross_entropy(real_labels, fake_output)

    # Compute gradients and update generator
    gradients_of_generator = gen_tape.gradient(g_loss, generator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))

    return d_loss, g_loss

# Training loop
for epoch in range(num_epochs):
    for i, image_batch in enumerate(train_dataset):
        d_loss, g_loss = train_step(image_batch)

        # Output training stats
        if i % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}] Batch {i}/{len(train_dataset)} '
                  f'Loss D: {d_loss:.4f}, Loss G: {g_loss:.4f}')

        # Save losses for plotting later
        G_losses.append(g_loss.numpy())
        D_losses.append(d_loss.numpy())

        # Save generated images periodically
        if (iters % 500 == 0) or ((epoch == num_epochs - 1) and (i == len(train_dataset) - 1)):
            generated_images = generator(fixed_noise, training=False)
            img_list.append(generated_images)
        iters += 1

# ========================
# 7. Visualize the Results
# ========================

import matplotlib.pyplot as plt
import matplotlib.animation as animation

# Plot the losses
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses, label="Generator")
plt.plot(D_losses, label="Discriminator")
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.legend()
plt.savefig('dcgan_results/loss_curves.png')
plt.show()

# Animation showing the improvements of the generator
fig = plt.figure(figsize=(8,8))
plt.axis("off")

ims = []
for images in img_list:
    # Rescale images to [0,1] for display
    images = (images + 1) / 2.0
    grid = tf.concat([tf.concat([images[i*8 + j] for j in range(8)], axis=1) for i in range(8)], axis=0)
    ims.append([plt.imshow(grid.numpy(), animated=True)])

ani = animation.ArtistAnimation(fig, ims, interval=500, repeat_delay=1000, blit=True)

# Save the animation as a GIF file
ani.save('dcgan_results/generation_animation.gif', writer='pillow')
print("Animation saved as 'dcgan_results/generation_animation.gif'.")

# Display real images
sample_real_images = next(iter(train_dataset))
sample_real_images = (sample_real_images + 1) / 2.0  # Rescale to [0,1]
grid_real = tf.concat([tf.concat([sample_real_images[i*8 + j] for j in range(8)], axis=1) for i in range(8)], axis=0)
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Real Images")
plt.imshow(grid_real.numpy())
plt.savefig('dcgan_results/real_images.png')
plt.show()

# Display generated images from the last epoch
generated_images = generator(fixed_noise, training=False)
generated_images = (generated_images + 1) / 2.0  # Rescale to [0,1]
grid_fake = tf.concat([tf.concat([generated_images[i*8 + j] for j in range(8)], axis=1) for i in range(8)], axis=0)
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Fake Images")
plt.imshow(grid_fake.numpy())
plt.savefig(f'dcgan_results/fake_images_epoch_{num_epochs}.png')
plt.show()

# ========================
# 8. Save the Trained Generator Model
# ========================

# Save the trained generator model
generator.save('generator.h5')
print("Generator model saved as 'generator.h5'.")
